In [4]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# -------- إعدادات --------
WIDE_PATH   = "Equites_Close_Daily.csv"   # ملف wide (كل عمود = يوم، الصف = شركة)
CHANGE_PATH = "clean.csv"                 # ملف طويل يحتوي عمود "التغيير"
WINDOW_LEN  = 20
TRAIN_RATIO = 0.8
EPOCHS      = 12
BATCH_SIZE  = 128

# -------- دوال --------
def build_sequences_with_idx(series_vals, window):
    """
    يبني تسلسلات الإدخال والإخراج ويُرجع أيضًا فهارس الأهداف (t) التي تم الاحتفاظ بها،
    حتى نربطها لاحقًا بقيم التغير بحسب التاريخ.
    """
    X, y, kept_idx = [], [], []
    for t in range(window, len(series_vals)):
        feat = series_vals[t-window:t]
        target = series_vals[t]
        if np.isnan(feat).any() or np.isnan(target):
            continue
        X.append(feat)
        y.append(target)
        kept_idx.append(t)
    if not X:
        return np.empty((0, window, 1), dtype="float32"), np.empty((0,), dtype="float32"), np.array([], dtype=int)
    X = np.array(X, dtype="float32")[:, :, None]
    y = np.array(y, dtype="float32")
    return X, y, np.array(kept_idx, dtype=int)

# -------- تحميل wide (إغلاقات) --------
wide = pd.read_csv(WIDE_PATH, encoding="utf-8-sig").set_index("اسم الشركة")
# تحويل القيم لأرقام
wide = wide.apply(pd.to_numeric, errors="coerce")
# توحيد تنسيق أعمدة التواريخ لصيغة YYYY-MM-DD
wide_cols_dt = pd.to_datetime(wide.columns, errors="coerce")
wide.columns = wide_cols_dt.strftime("%Y-%m-%d")
date_cols = list(wide.columns)

# -------- تحميل clean.csv (طويل) وأخذ عمود "التغيير" --------
change_long = pd.read_csv(CHANGE_PATH, encoding="utf-8-sig")

col_company = "اسم الشركة"
col_date    = "التاريخ"
col_change  = "التغيير"       # <-- التعديل هنا

# التحقق من الأعمدة
for c in [col_company, col_date, col_change]:
    if c not in change_long.columns:
        raise ValueError(f"عمود '{c}' غير موجود في clean.csv. الأعمدة المتاحة: {list(change_long.columns)}")

# تجهيز التواريخ والقيم
change_long[col_date] = pd.to_datetime(change_long[col_date], errors="coerce").dt.strftime("%Y-%m-%d")
change_long[col_change] = pd.to_numeric(change_long[col_change], errors="coerce")

# قاموس lookup: شركة -> {تاريخ: تغيير}
change_maps = {}
for comp, dfc in change_long.groupby(col_company):
    ser = (
        dfc.dropna(subset=[col_date])[[col_date, col_change]]
           .dropna()
           .drop_duplicates(subset=[col_date])
           .set_index(col_date)[col_change]
    )
    change_maps[comp] = ser.to_dict()

# -------- إعداد بيانات التدريب --------
X_tr_list, y_tr_list = [], []
X_te_list, y_te_list = [], []
te_mu_list, te_sigma_list = [], []
y_change_te_list = []      # لتقييم التغير من clean.csv بمحاذاة X_test
next_day_preds = []

for comp, row in wide.iterrows():
    vals = row.values.astype("float64")
    if np.sum(~np.isnan(vals)) < WINDOW_LEN + 2:
        continue

    X_all, y_all, kept_idx = build_sequences_with_idx(vals, WINDOW_LEN)
    if len(y_all) < 3:
        continue

    split = max(1, int(len(y_all) * TRAIN_RATIO))
    X_tr_c, y_tr_c = X_all[:split], y_all[:split]
    X_te_c, y_te_c = X_all[split:], y_all[split:]
    kept_idx_te = kept_idx[split:]

    if len(y_te_c) == 0:
        continue

    # تطبيع بناءً على train
    train_series_vals = np.concatenate([X_tr_c.reshape(-1), y_tr_c.reshape(-1)])
    mu = np.nanmean(train_series_vals)
    sigma = np.nanstd(train_series_vals)
    if not np.isfinite(mu) or sigma <= 1e-12:
        continue

    X_tr_c_norm = (X_tr_c - mu) / sigma
    y_tr_c_norm = (y_tr_c - mu) / sigma
    X_te_c_norm = (X_te_c - mu) / sigma
    y_te_c_norm = (y_te_c - mu) / sigma

    X_tr_list.append(X_tr_c_norm)
    y_tr_list.append(y_tr_c_norm)
    X_te_list.append(X_te_c_norm)
    y_te_list.append(y_te_c_norm)

    te_mu_list  += [mu] * len(y_te_c_norm)
    te_sigma_list += [sigma] * len(y_te_c_norm)

    # جلب قيم "التغيير" من clean.csv لنفس أيام الاختبار
    if comp in change_maps:
        mp = change_maps[comp]
        y_change_all = np.array([mp.get(date_cols[t], np.nan) for t in kept_idx], dtype="float64")
        y_change_te_list.append(y_change_all[split:])
    else:
        y_change_te_list.append(np.full(len(y_te_c), np.nan, dtype="float64"))

    # نافذة اليوم التالي
    last_window_raw = vals[-WINDOW_LEN:]
    if np.isnan(last_window_raw).any():
        next_day_preds.append({"اسم الشركة": comp, "next_day_pred": np.nan})
    else:
        last_window_norm = ((last_window_raw - mu) / sigma).astype("float32").reshape(1, WINDOW_LEN, 1)
        next_day_preds.append({
            "اسم الشركة": comp,
            "last_window_norm": last_window_norm,
            "mu": mu,
            "sigma": sigma,
            "yesterday": last_window_raw[-1]
        })

# دمج جميع الشركات
X_train = np.concatenate(X_tr_list, axis=0)
y_train = np.concatenate(y_tr_list, axis=0)
X_test  = np.concatenate(X_te_list, axis=0)
y_test  = np.concatenate(y_te_list, axis=0)
te_mu   = np.array(te_mu_list, dtype="float64")
te_sig  = np.array(te_sigma_list, dtype="float64")
y_change_test_from_file = np.concatenate(y_change_te_list, axis=0) if y_change_te_list else np.array([], dtype="float64")

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape,  y_test.shape)
print("Test changes from file:", y_change_test_from_file.shape)

# -------- RNN --------
tf.keras.backend.clear_session()
model = models.Sequential([
    layers.Input(shape=(WINDOW_LEN, 1)),
    layers.LSTM(64, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(32),
    layers.Dense(1)
])

model.compile(optimizer="adam", loss="mse")
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    shuffle=True
)

# -------- تقييم (على السعر) --------
y_hat_norm = model.predict(X_test, verbose=0).reshape(-1)
y_hat = y_hat_norm * te_sig + te_mu
y_true = y_test * te_sig + te_mu

R2   = r2_score(y_true, y_hat)
MAE  = mean_absolute_error(y_true, y_hat)
RMSE = float(np.sqrt(mean_squared_error(y_true, y_hat)))

print("\n=== Global Test Metrics (Close Level) ===")
print(f"R²   = {R2:.4f}")
print(f"MAE  = {MAE:.4f}")
print(f"RMSE = {RMSE:.4f}")

# -------- التغير (Δ) المُشتق من السعر (مرجعي) --------
prev_close = (X_test[:, -1, 0] * te_sig + te_mu)
true_change_from_price = y_true - prev_close
pred_change_from_price = y_hat  - prev_close

R2_change_price   = r2_score(true_change_from_price, pred_change_from_price)
MAE_change_price  = mean_absolute_error(true_change_from_price, pred_change_from_price)
RMSE_change_price = float(np.sqrt(mean_squared_error(true_change_from_price, pred_change_from_price)))

print("\n=== Change (Δ Close) Metrics — Derived from Close ===")
print(f"R² (Δ from price)   = {R2_change_price:.4f}")
print(f"MAE (Δ from price)  = {MAE_change_price:.4f}")
print(f"RMSE (Δ from price) = {RMSE_change_price:.4f}")

# -------- تقييم على التغيير من clean.csv --------
if y_change_test_from_file.size == X_test.shape[0]:
    mask_file = np.isfinite(y_change_test_from_file) & np.isfinite(pred_change_from_price)
    if mask_file.sum() > 0:
        R2_change_file   = r2_score(y_change_test_from_file[mask_file], pred_change_from_price[mask_file])
        MAE_change_file  = mean_absolute_error(y_change_test_from_file[mask_file], pred_change_from_price[mask_file])
        RMSE_change_file = float(np.sqrt(mean_squared_error(y_change_test_from_file[mask_file], pred_change_from_price[mask_file])))

        print("\n=== Change Metrics — From clean.csv (عمود 'التغيير') ===")
        print(f"R² (clean.csv)   = {R2_change_file:.4f}")
        print(f"MAE (clean.csv)  = {MAE_change_file:.6f}")
        print(f"RMSE (clean.csv) = {RMSE_change_file:.6f}")
    else:
        print("\n=== Change Metrics — From clean.csv ===")
        print("No valid samples after filtering NaNs/infs in clean.csv.")
else:
    print("\n[Warning] clean.csv alignment mismatch: "
          f"len(test changes)={y_change_test_from_file.size} vs X_test={X_test.shape[0]}")

# -------- (اختياري) نسب تغير آمنة + sMAPE للتغير المُشتق من السعر --------
eps = 1e-6
valid_pct = np.isfinite(prev_close) & (np.abs(prev_close) > eps)
true_change_pct = np.full_like(true_change_from_price, np.nan, dtype="float64")
pred_change_pct = np.full_like(pred_change_from_price, np.nan, dtype="float64")
true_change_pct[valid_pct] = true_change_from_price[valid_pct] / prev_close[valid_pct]
pred_change_pct[valid_pct] = pred_change_from_price[valid_pct] / prev_close[valid_pct]

mask_pct = np.isfinite(true_change_pct) & np.isfinite(pred_change_pct)
if mask_pct.sum() > 0:
    R2_change_pct   = r2_score(true_change_pct[mask_pct], pred_change_pct[mask_pct])
    MAE_change_pct  = mean_absolute_error(true_change_pct[mask_pct], pred_change_pct[mask_pct])
    RMSE_change_pct = float(np.sqrt(mean_squared_error(true_change_pct[mask_pct], pred_change_pct[mask_pct])))

    smape = np.mean(
        2.0 * np.abs(pred_change_from_price[mask_pct] - true_change_from_price[mask_pct]) /
        (np.abs(pred_change_from_price[mask_pct]) + np.abs(true_change_from_price[mask_pct]) + eps)
    )

    print("\n=== Change Percentage Metrics (filtered, from price) ===")
    print(f"R² (Δ%)   = {R2_change_pct:.4f}")
    print(f"MAE (Δ%)  = {MAE_change_pct:.6f}")
    print(f"RMSE (Δ%) = {RMSE_change_pct:.6f}")
    print(f"sMAPE (Δ) = {smape:.6f}")
else:
    print("\n=== Change Percentage Metrics (filtered, from price) ===")
    print("No valid samples for percentage metrics (prev_close too small/invalid).")

# -------- توقع + توصية --------
pred_rows = []
for rec in next_day_preds:
    comp = rec["اسم الشركة"]
    if "last_window_norm" not in rec:
        pred_rows.append({"اسم الشركة": comp, "yesterday": np.nan, "predicted": np.nan, "recommendation": "N/A"})
        continue
    pred_norm = model.predict(rec["last_window_norm"], verbose=0).reshape(-1)[0]
    pred = pred_norm * rec["sigma"] + rec["mu"]
    yesterday = rec["yesterday"]
    recommendation = "Buy" if pred < yesterday else "Don’t Buy"
    pred_rows.append({
        "اسم الشركة": comp,
        "yesterday": float(yesterday),
        "predicted": float(pred),
        "recommendation": recommendation
    })

pred_df = pd.DataFrame(pred_rows)
print("\nSample predictions with recommendations:")
print(pred_df.head(10))

# -------- مجلد الحفظ --------
OUT_DIR = "out"
os.makedirs(OUT_DIR, exist_ok=True)

# -------- حفظ الموديل --------
model_path = os.path.join(OUT_DIR, "rnn_model_with_norm.h5")
model.save(model_path)
print(f"✅ Saved model: {model_path}")

# -------- حفظ النتائج --------
csv_path = os.path.join(OUT_DIR, "rnn_recommendations.csv")
pred_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"✅ Saved recommendations: {csv_path}")

Train: (98200, 20, 1) (98200,)
Test : (24600, 20, 1) (24600,)
Test changes from file: (24600,)
Epoch 1/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 23s 26ms/step - loss: 0.0296 - val_loss: 0.1453
Epoch 2/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - loss: 0.0104 - val_loss: 0.1109
Epoch 3/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - loss: 0.0093 - val_loss: 0.0918
Epoch 4/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - loss: 0.0091 - val_loss: 0.0870
Epoch 5/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 20s 26ms/step - loss: 0.0089 - val_loss: 0.0793
Epoch 6/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - loss: 0.0090 - val_loss: 0.0670
Epoch 7/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - loss: 0.0088 - val_loss: 0.0829
Epoch 8/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - loss: 0.0087 - val_loss: 0.0750
Epoch 9/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - loss: 0.0087 - val_loss: 0.0931
Epoch 10/12
768/768 ━━━━━━━━━━━━━━━━━━━━ 19s 25ms/step - loss: 0.0087 - val_loss: 0.0812
Epoch 11/12
768/768 ━━━━


Sample predictions with recommendations:
                                اسم الشركة  yesterday   predicted  \
0                 اعمار المدينة الاقتصادية      17.23   17.239176   
1                     البنك الاهلي السعودي      38.65   38.843105   
2                      البنك السعودي الاول      37.90   37.915913   
3                    البنك السعودي الفرنسي      19.30   19.335321   
4                  البنك السعودي للاستثمار      12.76   12.737811   
5                      البنك العربي الوطني      18.97   18.989807   
6                 الشركة التعاونية للتامين     130.20  107.352341   
7  الشركة الخليجية العامة للتامين التعاوني      13.17   13.272333   
8           الشركة السعودية لاعادة التامين      18.60   18.531908   
9            الشركة السعودية لانابيب الصلب      36.30   35.977356   

  recommendation  
0      Don’t Buy  
1      Don’t Buy  
2      Don’t Buy  
3      Don’t Buy  
4            Buy  
5      Don’t Buy  
6            Buy  
7      Don’t Buy  
8            Buy  
9       

In [5]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models
import os

# =================== إعدادات ===================
WIDE_PATH    = "Equites_Close_Daily.csv"   # wide: الصف = اسم الشركة، الأعمدة = تواريخ، القيم = إقفال
CLEAN_PATH   = "clean.csv"                 # long: يحتوي عمود "التغيير" اليومي
WINDOW_LEN   = 20
TRAIN_RATIO  = 0.8
EPOCHS       = 12
BATCH_SIZE   = 128
EPS          = 1e-6

# =================== دوال مساعدة ===================
def build_sequences(series_vals, window):
    X, y = [], []
    for t in range(window, len(series_vals)):
        feat = series_vals[t-window:t]
        target = series_vals[t]
        if np.isnan(feat).any() or np.isnan(target):
            continue
        X.append(feat)
        y.append(target)
    if not X:
        return np.empty((0, window, 1), dtype="float32"), np.empty((0,), dtype="float32")
    X = np.array(X, dtype="float32")[:, :, None]
    y = np.array(y, dtype="float32")
    return X, y

def series_from_long(df_long, company_col, date_col, value_col):
    """حوّل long إلى Series: index=date(str YYYY-MM-DD), values=value (float)."""
    tmp = df_long[[company_col, date_col, value_col]].copy()
    tmp[date_col] = pd.to_datetime(tmp[date_col], errors="coerce").dt.strftime("%Y-%m-%d")
    tmp[value_col] = pd.to_numeric(tmp[value_col], errors="coerce")
    tmp = tmp.dropna(subset=[date_col, value_col])
    return {c: g.drop_duplicates(subset=[date_col]).set_index(date_col)[value_col].sort_index()
            for c, g in tmp.groupby(company_col)}

# =================== تحميل البيانات ===================
# Wide prices (Close)
wide = pd.read_csv(WIDE_PATH, encoding="utf-8-sig").set_index("اسم الشركة")
wide = wide.apply(pd.to_numeric, errors="coerce")
wide.columns = pd.to_datetime(wide.columns, errors="coerce").strftime("%Y-%m-%d")

# Long changes
clean = pd.read_csv(CLEAN_PATH, encoding="utf-8-sig")
COL_COMPANY = "اسم الشركة"
COL_DATE    = "التاريخ"
COL_CHANGE  = "التغيير"   # لو أردت النسبة استخدم '% التغيير'

change_map = series_from_long(clean, COL_COMPANY, COL_DATE, COL_CHANGE)

# سنحتاج أسعار الأمس لإعادة بناء السعر من Δ_pred
# نبني أيضاً Series للأسعار لكل شركة (من wide)
price_map = {comp: row.dropna().astype(float) for comp, row in wide.iterrows()}

# =================== بناء بيانات التدريب على Δ ===================
X_tr_list, y_tr_list = [], []
X_te_list, y_te_list = [], []
te_mu_list, te_sig_list = [], []

# للاحتفاظ بمحاذاة الأمس لإعادة بناء السعر، ولحساب Hit Rate على الاختبار
prev_close_test_all = []
true_close_test_all = []   # لإعادة بناء السعر الحقيقي أيضاً إن أحببنا
comp_windows_for_next_pred = []  # لكل شركة: آخر نافذة Δ + سعر الأمس

for comp, change_ser in change_map.items():
    # التواريخ المشتركة بين تغييرات الشركة وأسعارها (عشان نقدر نرجّع للسعر)
    if comp not in price_map:
        continue
    price_ser = price_map[comp]
    common_dates = change_ser.index.intersection(price_ser.index)
    if len(common_dates) < WINDOW_LEN + 2:
        continue

    # مصفوفة Δ مرتبة زمنياً
    delta_vals = change_ser.loc[common_dates].values.astype("float64")
    # أسعار مطابقة (لنستخدم P_{t-1} لاحقاً)
    close_vals = price_ser.loc[common_dates].values.astype("float64")

    # بناء تسلسلات على Δ
    X_all, y_all = build_sequences(delta_vals, WINDOW_LEN)
    if len(y_all) < 3:
        continue

    # تقسيم
    split = max(1, int(len(y_all) * TRAIN_RATIO))
    X_tr_c, y_tr_c = X_all[:split], y_all[:split]
    X_te_c, y_te_c = X_all[split:], y_all[split:]
    if len(y_te_c) == 0:
        continue

    # تطبيع على Δ (train فقط)
    tr_vec = np.concatenate([X_tr_c.reshape(-1), y_tr_c.reshape(-1)])
    mu, sig = np.nanmean(tr_vec), np.nanstd(tr_vec)
    if not np.isfinite(mu) or sig <= 1e-12:
        continue

    X_tr_c_n = (X_tr_c - mu) / sig
    y_tr_c_n = (y_tr_c - mu) / sig
    X_te_c_n = (X_te_c - mu) / sig
    y_te_c_n = (y_te_c - mu) / sig

    X_tr_list.append(X_tr_c_n); y_tr_list.append(y_tr_c_n)
    X_te_list.append(X_te_c_n); y_te_list.append(y_te_c_n)
    te_mu_list += [mu] * len(y_te_c_n)
    te_sig_list += [sig] * len(y_te_c_n)

    # prev_close لكل عينة اختبار = سعر اليوم السابق (أخر عنصر نافذة الأسعار الموافقة)
    # نبني prev_close_test_all بمحاذاة عينات الاختبار
    # ملاحظة: نافذة Δ بطول WINDOW_LEN تبدأ من t-WINDOW_LEN .. t-1
    # بينما prev_close = P_{t-1}
    # لنسترجع P_{t-1} نأخذ close_vals مع تعويض نفس عمليات الإسقاط التي قام بها build_sequences
    prev_close_all = []
    true_close_all = []
    for t in range(WINDOW_LEN, len(close_vals)):
        if t < split + WINDOW_LEN:  # هذه عينات train
            continue
        prev_close_all.append(close_vals[t-1])
        true_close_all.append(close_vals[t])
    prev_close_test_all.extend(prev_close_all)
    true_close_test_all.extend(true_close_all)

    # تحضير تنبؤ اليوم التالي لكل شركة (آخر نافذة Δ)
    last_delta_window = delta_vals[-WINDOW_LEN:]
    if not np.isnan(last_delta_window).any():
        comp_windows_for_next_pred.append({
            "اسم الشركة": comp,
            "last_window_norm": ((last_delta_window - mu) / sig).astype("float32").reshape(1, WINDOW_LEN, 1),
            "mu": mu, "sig": sig,
            "yesterday_close": close_vals[-1]
        })

# دمج
X_train = np.concatenate(X_tr_list, axis=0) if X_tr_list else np.empty((0, WINDOW_LEN, 1), dtype="float32")
y_train = np.concatenate(y_tr_list, axis=0) if y_tr_list else np.empty((0,), dtype="float32")
X_test  = np.concatenate(X_te_list, axis=0) if X_te_list else np.empty((0, WINDOW_LEN, 1), dtype="float32")
y_test  = np.concatenate(y_te_list, axis=0) if y_te_list else np.empty((0,), dtype="float32")
te_mu   = np.array(te_mu_list, dtype="float64")
te_sig  = np.array(te_sig_list, dtype="float64")
prev_close_test_all = np.array(prev_close_test_all, dtype="float64")
true_close_test_all = np.array(true_close_test_all, dtype="float64")

print("Train Δ:", X_train.shape, y_train.shape)
print("Test  Δ:", X_test.shape,  y_test.shape)

# =================== نموذج RNN على Δ ===================
tf.keras.backend.clear_session()
model = models.Sequential([
    layers.Input(shape=(WINDOW_LEN, 1)),
    layers.LSTM(64, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(32),
    layers.Dense(1)
])
model.compile(optimizer="adam", loss="mse")

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    shuffle=True
)

# =================== تقييم على Δ ===================
y_hat_n = model.predict(X_test, verbose=0).reshape(-1)
y_pred_delta = y_hat_n * te_sig + te_mu     # التغير المتنبأ
y_true_delta = y_test   * te_sig + te_mu     # التغير الحقيقي

R2_delta   = r2_score(y_true_delta, y_pred_delta)
MAE_delta  = mean_absolute_error(y_true_delta, y_pred_delta)
RMSE_delta = float(np.sqrt(mean_squared_error(y_true_delta, y_pred_delta)))

# اتجاه الحركة (Hit Rate)
hit_rate = np.mean(np.sign(y_true_delta) == np.sign(y_pred_delta)) if y_true_delta.size else np.nan

print("\n=== Test Metrics on Δ (trained on Δ) ===")
print(f"R² (Δ)   = {R2_delta:.4f}")
print(f"MAE (Δ)  = {MAE_delta:.6f}")
print(f"RMSE (Δ) = {RMSE_delta:.6f}")
print(f"Hit Rate = {hit_rate:.3f}")

# (اختياري) نسب التغير الآمنة
valid_pct = np.abs(prev_close_test_all) > EPS
if valid_pct.sum() == y_true_delta.size == y_pred_delta.size:
    true_pct = y_true_delta[valid_pct] / prev_close_test_all[valid_pct]
    pred_pct = y_pred_delta[valid_pct] / prev_close_test_all[valid_pct]
    if true_pct.size > 0:
        R2_pct   = r2_score(true_pct, pred_pct)
        MAE_pct  = mean_absolute_error(true_pct, pred_pct)
        RMSE_pct = float(np.sqrt(mean_squared_error(true_pct, pred_pct)))
        smape    = np.mean(2.0*np.abs(pred_pct-true_pct)/(np.abs(pred_pct)+np.abs(true_pct)+EPS))
        print("\n=== Δ% Metrics (filtered) ===")
        print(f"R² (Δ%)   = {R2_pct:.4f}")
        print(f"MAE (Δ%)  = {MAE_pct:.6f}")
        print(f"RMSE (Δ%) = {RMSE_pct:.6f}")
        print(f"sMAPE     = {smape:.6f}")

# =================== إعادة بناء السعر + توصيات ===================
# السعر المتنبأ لعَيّنات الاختبار (للمراجعة): P_hat = P_{t-1} + Δ_hat
if prev_close_test_all.size == y_pred_delta.size:
    price_pred_test = prev_close_test_all + y_pred_delta
    price_true_test = true_close_test_all
    R2_price   = r2_score(price_true_test, price_pred_test)
    MAE_price  = mean_absolute_error(price_true_test, price_pred_test)
    RMSE_price = float(np.sqrt(mean_squared_error(price_true_test, price_pred_test)))
    print("\n=== Reconstructed Close (from Δ) on Test ===")
    print(f"R² (Close)   = {R2_price:.4f}")
    print(f"MAE (Close)  = {MAE_price:.6f}")
    print(f"RMSE (Close) = {RMSE_price:.6f}")

# توقع اليوم التالي لكل شركة
pred_rows = []
for rec in comp_windows_for_next_pred:
    comp = rec["اسم الشركة"]
    yhat_n = model.predict(rec["last_window_norm"], verbose=0).reshape(-1)[0]
    delta_pred = yhat_n * rec["sig"] + rec["mu"]     # Δ_pred للغد
    yesterday  = rec["yesterday_close"]
    tomorrow_pred_price = float(yesterday + delta_pred)
    recommendation = "Buy" if delta_pred > 0 else "Don’t Buy"
    pred_rows.append({
        "اسم الشركة": comp,
        "yesterday": float(yesterday),
        "predicted_delta": float(delta_pred),
        "predicted_close": tomorrow_pred_price,
        "recommendation": recommendation
    })

pred_df = pd.DataFrame(pred_rows).sort_values("اسم الشركة")
print("\nSample next-day predictions:")
print(pred_df.head(10))

# =================== الحفظ ===================
OUT_DIR = "out"
os.makedirs(OUT_DIR, exist_ok=True)

model_path = os.path.join(OUT_DIR, "rnn_on_change.h5")
model.save(model_path)
print(f"✅ Saved model: {model_path}")

csv_path = os.path.join(OUT_DIR, "nextday_from_change.csv")
pred_df.to_csv(csv_path, index=False, encoding="utf-8-sig")
print(f"✅ Saved next-day predictions: {csv_path}")

Train Δ: (33388, 20, 1) (33388,)
Test  Δ: (8364, 20, 1) (8364,)
Epoch 1/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - loss: 1.0062 - val_loss: 2.4925
Epoch 2/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 1.0051 - val_loss: 2.4944
Epoch 3/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 1.0047 - val_loss: 2.4872
Epoch 4/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 1.0047 - val_loss: 2.4872
Epoch 5/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 1.0045 - val_loss: 2.4811
Epoch 6/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 1.0045 - val_loss: 2.4830
Epoch 7/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 1.0035 - val_loss: 2.4808
Epoch 8/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 1.0022 - val_loss: 2.4808
Epoch 9/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 1.0004 - val_loss: 2.4823
Epoch 10/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 0.9992 - val_loss: 2.4914
Epoch 11/12
261/261 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 0.


Sample next-day predictions:
                                 اسم الشركة  yesterday  predicted_delta  \
0                     البنك السعودي الفرنسي      19.30         0.010686   
1                       البنك العربي الوطني      18.97        -0.023904   
2  الشركة السعودية لخدمات السيارات والمعدات      63.68        -0.275620   
3              الشركة السعودية لصناعة الورق      40.32         0.054115   
4          الشركة السعودية للتنمية الصناعية      33.69         0.027845   
5         الشركة السعودية للصادرات الصناعية       2.83        -0.000215   
6         الشركة السعودية للصناعات المتطورة      28.65         0.022825   
7          الشركة السعودية للطباعة والتغليف      15.56         0.002943   
8                  الشركة السعودية للكهرباء      18.98         0.009626   
9             الشركة السعودية للنقل الجماعي      20.74         0.029127   

   predicted_close recommendation  
0        19.310686            Buy  
1        18.946096      Don’t Buy  
2        63.404380      Don’t Buy  
